In [1]:
import pystan
import pandas as pd

%load_ext jupyterstan
%matplotlib inline

## 数据准备

在同一个高斯随机场中，用不同的发射信号源（发射功率，坐标值），产生感知节点的接收信号强度RSS。

## 参数估计模型

In [8]:
%%stan parameter_estimate

data {
  int<lower=1> N_data; // 训练数据集大小
  int<lower=1> N_nodes; // 感知节点个数
  vector[N_nodes] RSS[N_data];
  row_vector[2] sensor_location[N_nodes];
  real trans_power[N_data];
  row_vector[2] trans_location[N_data];
}

transformed data {
  vector[N_nodes] distance_sensor_trans[N_data];
  for (j in 1:N_data) {
    for (i in 1:N_nodes){
      distance_sensor_trans[j][i] = distance(sensor_location[i], trans_location[j]);
    }  
  }
}

parameters {
  real<lower=0> sigma_epsilon; //测量误差
  real sigma; // 空间方差
  real fai;  // matern 协方差函数
  real gammma; //自由空间传播损耗因子
}

transformed parameters {

}

model {
  vector[N_nodes] mu[N_data];
 
  matrix[N_nodes, N_nodes] cov;
  
  /* 生成一个单位矩阵 */
  vector[N_nodes] i_vec = rep_vector(1.0, N_nodes);
  matrix[N_nodes, N_nodes] unit_matrix = diag_matrix(i_vec);

  
  /* 先验分布 */
  sigma_epsilon ~ inv_gamma(2, 10);
  sigma ~ inv_gamma(2, 10);
  fai ~ uniform(1, 100);
  
  for (i in 1:N_data){
    mu[i] = trans_power[i] - 10 * gammma * log10(distance_sensor_trans[i]);
  }
  
  cov = cov_exp_quad(sensor_location, sensor_location, sigma, 1/sqrt(2*fai) )+ sigma_epsilon * unit_matrix;
  
  RSS ~ multi_normal(mu,cov);
}

INFO:pystan:COMPILING THE C++ CODE FOR MODEL parameter_estimate_9a7ea51f517a8fc335c35226c3e95a34 NOW.


Creating pystan model & assigning it to variable name "parameter_estimate".
Stan options:
 {'model_name': 'parameter_estimate', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}
StanModel now available as variable "parameter_estimate"!
Compilation took a minute.


In [2]:
pystan.stan?